In [1]:
%pip install torch transformers datasets tqdm
!mkdir checkpoints

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%pip install datasets

In [2]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader
import json


# Load necessary packages
from dataclasses import dataclass
from typing import List, Tuple

from torch.utils.data import Dataset
from transformers import PreTrainedTokenizerFast
from datasets import load_dataset

import torch.nn as nn
from torch.optim import Optimizer, AdamW
from tqdm.notebook import tqdm

In [3]:
# from datasets import Dataset
import pickle

In [4]:
from transformers import pipeline
# from transformers.pipelines.pt_utils import KeyDataset

In [5]:
from datasets import load_dataset

# ds = load_dataset("ChilleD/StrategyQA")

In [10]:
with open('./data/train.json', 'r') as file:
    train_data = json.load(file)
# with open('/kaggle/input/strategyqa/train.json', 'r') as file:
    # train_data = json.load(file)

# Print the data
print(len(train_data))

2061


In [11]:
with open('./data/dev.json', 'r') as file:
    validation_data = json.load(file)
# with open('/kaggle/input/strategyqa/dev.json', 'r') as file:
    # validation_data = json.load(file)

# Print the data
print(len(validation_data))

229


In [12]:
train_data[0]

{'qid': '2bc9c4f9c19c167187f2',
 'term': 'Genghis Khan',
 'description': 'founder and first Great Khan of the Mongol Empire',
 'question': 'Are more people today related to Genghis Khan than Julius Caesar?',
 'answer': True,
 'facts': ['Julius Caesar had three children.',
  'Genghis Khan had sixteen children.',
  'Modern geneticists have determined that  out of every 200 men today has DNA that can be traced to Genghis Khan.'],
 'decomposition': ['How many kids did Julius Caesar have?',
  'How many kids did Genghis Khan have?',
  'Is #2 greater than #1?'],
 'evidence': [[[['Caesarion-2', 'Julia (daughter of Caesar)-1']],
   [['Alakhai Bekhi-1', 'Tolui-1'], 'no_evidence'],
   ['operation']],
  [[['Julius Caesar-75']], [['Genghis Khan-17']], ['operation']],
  [[['Gaius Julius Caesar-7']],
   [['Genghis Khan-15'], 'no_evidence'],
   ['no_evidence', 'operation']]]}

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [18]:
def postprocess_answer2(answer):
    answer = answer.strip().lower()

    if "final answer: no" in answer:
        return "no"
    elif "final answer: yes" in answer:
        return "yes"

    # Positive signals for "Yes"
    yes_patterns = [
        "yes", "indeed", "correct", "true", "certainly", "definitely", "of course"
    ]

    # Negative signals for "No"
    no_patterns = [
        "no", "not", "incorrect", "false", "impossible", "unlikely", "cannot", "never"
    ]

    # Uncertainty signals (default to "no" or "unknown" depending on strategy)
    uncertain_patterns = [
        "not sure", "unsure", "uncertain", "maybe", "possibly", "potentially", "it depends"
    ]

    # Check for direct yes/no answers
    if any(pattern in answer for pattern in yes_patterns):
        return "yes"
    elif any(pattern in answer for pattern in no_patterns):
        return "no"
    elif any(pattern in answer for pattern in uncertain_patterns):
        return "unknown"  # or "no" if you want a hard yes/no

    # Fallback: try to map using first sentence heuristic
    first_sentence = answer.split(".")[0]
    if "yes" in first_sentence:
        return "yes"
    elif "no" in first_sentence:
        return "no"

    # Default to "unknown" if nothing matches
    return "unknown"


In [19]:
def postprocess_answer3(answer):
    answer = answer.strip().lower()

    if "**final answer**: no" in answer or "**conclusion**: no" in answer:
        return "no"
    elif "**final answer**: yes" in answer or "**conclusion**: yes" in answer:
        return "yes"

    # Positive signals for "Yes"
    yes_patterns = [
        "yes", "indeed", "correct", "true", "certainly", "definitely", "of course"
    ]

    # Negative signals for "No"
    no_patterns = [
        "no", "not", "incorrect", "false", "impossible", "unlikely", "cannot", "never"
    ]

    # Uncertainty signals (default to "no" or "unknown" depending on strategy)
    uncertain_patterns = [
        "not sure", "unsure", "uncertain", "maybe", "possibly", "potentially", "it depends"
    ]

    # Check for direct yes/no answers
    """
    if any(pattern in answer for pattern in yes_patterns):
        return "yes"
    elif any(pattern in answer for pattern in no_patterns):
        return "no"
    elif any(pattern in answer for pattern in uncertain_patterns):
        return "unknown"  # or "no" if you want a hard yes/no
    """
    yess = sum([pattern in answer for pattern in yes_patterns])
    nos = sum([pattern in answer for pattern in no_patterns])
    unks = sum([pattern in answer for pattern in uncertain_patterns])

    if yess > nos and yess > unks:
        return "yes"
    elif nos > yess and nos > unks:
        return "no"

    # Fallback: try to map using last sentence heuristic
    first_sentence = answer.split(".")[-1]
    if "yes" in first_sentence:
        return "yes"
    elif "no" in first_sentence:
        return "no"

    # Default to "unknown" if nothing matches
    return "unknown"


In [20]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer

preds = []
actuals = []

predTrue = []
predFalse = []

generated_ans = []

unknowns = []

model_name = "distilgpt2"

torch.manual_seed(2)

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [22]:
batch_size = 32


tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

prompt = """Let's think this through step by step. Decompose the question into a series of explicit sub-questions and answer each of them.
Example 1:
Question: Are more people today related to Genghis Khan than Julius Caesar?
Decomposition:
1. How many kids did Julius Caesar have?
Julius Caesar had three children.
2. How many kids did Genghis Khan have?
Genghis Khan had sixteen children.
3. Is #2 greater than #1?
Final Answer: Yes

Example 2:
Question: Does a Generation Y member satisfy NYPD police officer age requirement?
Decomposition:
1. How old do you have to be to be an NYPD officer?
The NYPD has a minimum age requirement of 21.
2. How old are Generation Y members currently?
Members of Generation Y were born between 1980 and 1994.
3. Is #2 higher than #1?
Final Answer: Yes

Example 3:
Question: Is shrimp scampi definitely free of plastic?
Decomposition:
1. What protein is Shrimp scampi made out of?
Shrimp scampi is a dish made with shrimp.
2. What have #1 been found to contain?
Shrimp have been found to contain microplastics.
3. Are #2 free from plastic?
Microplastics are plastic material.
Final Answer: No

Question: """

preds = []
actuals = []

predTrue = []
predFalse = []

generated_ans = []

unknowns = []

# Split validation data into batches
for i in range(0, len(validation_data), batch_size):
    print(i)
    batch = validation_data[i:i + batch_size]
    prompts = [
        prompt + f"{d['question']}\nDecomposition:\n"
        for d in batch
    ]

    sequences = pipe(
        prompts,
        batch_size=batch_size,
        do_sample=True,
        top_k=10,
        max_new_tokens=100,
        return_full_text = False,
        pad_token_id=tokenizer.eos_token_id
    )

    for seq, d in zip(sequences, batch):
        tex = seq[0]["generated_text"]
        generated_ans.append(tex)
        ans = postprocess_answer2(tex)
        if ans == 'unknown':
            unknowns.append((tex, d["answer"]))
            continue
        if ans == 'yes':
            predTrue.append(tex)
        else:
            predFalse.append(tex)
        preds.append(ans == 'yes')
        actuals.append(d["answer"])


0
32
64
96
128
160
192
224


In [23]:
len(generated_ans)

229

In [24]:
import pickle

# Create a dictionary of variables to save
data_to_save = {
    'preds': preds,
    'actuals': actuals,
    'predTrue': predTrue,
    'predFalse': predFalse,
    'generated_ans': generated_ans,
    'unknowns': unknowns
}

# Save to file
with open('./data/saved_variables.pkl', 'wb') as f:
    pickle.dump(data_to_save, f)

In [25]:
sum([a==b for a,b in zip(preds, actuals)]) / len(preds)

0.4695121951219512

In [26]:
len(preds)

164

In [27]:
unknowns[:4]

[('1. Will the Albany in Georgia be able to maintain its population?\nThe Albany has one population.\n2. If a generation of people is older, what do you think is a good chance of becoming the first generation to have a chance of becoming the first generation to have a chance at becoming the first generation to have a chance at becoming the first generation to have a chance at gaining the first generation to have a chance at gaining the first generation to gain a chance at obtaining the first generation to',
  False),
 ("1. How many girls did Lil Jon's top ranked Billboard song a collaboration with a member of The Lox?\nThe Lox is a song made with a member of The Lox?\nThe Lox is a song made with a member of The Lox?\nThe Lox is a song made with a member of The Lox?\nThe Lox is a song made with a member of The Lox?\nThe Lox is a song made with a member of The",
  False),
 ('1. Why does the Swiss Guard fill the Virginia General Assembly chairs?\nThe Swiss Guard fills the Virginia General

In [28]:
with open('./data/saved_variables.pkl', 'rb') as f:
    data = pickle.load(f)

In [29]:
data['generated_ans'][1]

'1. Do I belong to a family who have been raised in a family in the United States?\nThe French family was raised in the United States in the 1950s and 1960s.\n2. Do I belong to a family who has been raised in a family in the United States?\nThe French family was raised in the 1970s and 1970s.\n3. Are #2 higher than #1?\nFinal Answer: Yes\nExample 4:\nQuestion: Is the language used'

In [30]:
prompt2 = """
Given the original question, the reasoning steps do not make sense. A better reasoning sequence would be
Reasoning:
1.
"""
prompt = f"Question: {validation_data[0]['question']}\nReasoning:\n" + data['generated_ans'][0] + prompt2
print(prompt)

Question: Will the Albany in Georgia reach a hundred thousand occupants before the one in New York?
Reasoning:
1. Will the Albany in Georgia be able to maintain its population?
The Albany has one population.
2. If a generation of people is older, what do you think is a good chance of becoming the first generation to have a chance of becoming the first generation to have a chance at becoming the first generation to have a chance at becoming the first generation to have a chance at gaining the first generation to have a chance at gaining the first generation to gain a chance at obtaining the first generation to
Given the original question, the reasoning steps do not make sense. A better reasoning sequence would be
Reasoning:
1.



In [31]:
batch_size = 64

preds = []
actuals = []

predTrue = []
predFalse = []

generated_ans = []

unknowns = []

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

prompt2 = """
Given the original question, the reasoning steps do not make sense. A better reasoning sequence would be
Reasoning:
1.
"""

# Split validation data into batches
for i in range(0, len(validation_data), batch_size):
    print(i)
    batch = validation_data[i:i + batch_size]
    prompts = [
        f"Question: {d['question']}\nReasoning:\n"
        for d in batch
    ]
    for j in range(i, min(len(validation_data), i+batch_size)):
        prompts[j-i] += data['generated_ans'][j] + prompt2

    sequences = pipe(
        prompts,
        batch_size=batch_size,
        do_sample=True,
        top_k=10,
        max_new_tokens=500,
        return_full_text = False,
        pad_token_id=tokenizer.eos_token_id
    )

    for seq, d in zip(sequences, batch):
        tex = seq[0]["generated_text"]
        generated_ans.append(tex)
        ans = postprocess_answer3(tex)
        if ans == 'unknown':
            unknowns.append((tex, d["answer"]))
            continue
        if ans == 'yes':
            predTrue.append(tex)
        else:
            predFalse.append(tex)
        preds.append(ans == 'yes')
        actuals.append(d["answer"])

# Create a dictionary of variables to save
data_to_save = {
    'preds': preds,
    'actuals': actuals,
    'predTrue': predTrue,
    'predFalse': predFalse,
    'generated_ans': generated_ans,
    'unknowns': unknowns
}

# Save to file
with open('./data/saved_variables_deepskeptic.pkl', 'wb') as f:
    pickle.dump(data_to_save, f)

0
64
128
192


In [32]:
sum([a==b for a,b in zip(preds, actuals)]) / len(preds)

0.5230769230769231